In [1]:
import os
import json
import pandas as pd
import scipy as sp
from sklearn.preprocessing import LabelEncoder 
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
import datetime
import pickle
from pprint import pprint
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import seaborn as sns
import gc

In [2]:
"""
#### User features ####
id: user_id_hash 16264937
user_recs
user_clicks 
user_target_recs
page_view_start_time
#######################

#### Item features ####
id: target_id_hash 95082
syndicator_id_hash 2809
campaign_id_hash 28052
target_item_taxonomy 66
placement_id_hash 1629
empiric_calibrated_recs
empiric_clicks
target_item_taxonomy 66
#######################

#### Context ####
publisher_id_hash 3
source_id_hash 264338
source_item_type 5
browser_platform 5
country_code 406
region 1965
os_family  7
day_of_week
time_of_day
gmt_offset 36
#################
"""

'\n#### User features ####\nid: user_id_hash 16264937\nuser_recs\nuser_clicks \nuser_target_recs\npage_view_start_time\n#######################\n\n#### Item features ####\nid: target_id_hash 95082\nsyndicator_id_hash 2809\ncampaign_id_hash 28052\ntarget_item_taxonomy 66\nplacement_id_hash 1629\nempiric_calibrated_recs\nempiric_clicks\ntarget_item_taxonomy 66\n#######################\n\n#### Context ####\npublisher_id_hash 3\nsource_id_hash 264338\nsource_item_type 5\nbrowser_platform 5\ncountry_code 406\nregion 1965\nos_family  7\nday_of_week\ntime_of_day\ngmt_offset 36\n#################\n'

# Better classifier
Build two tower model:
* one tower for user embeddings
* second tower for item embeddings
* for each record the embedding will be the concat of numeric features and embedding of a lookup table

In [2]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
NVIDIA GeForce GTX 1080 Ti


In [3]:
class CategoricalFeature:
    def __init__(self, name, vocabulary_size, embedding_size) -> None:
        # + 1 for unknown
        self.lookup_table = nn.Embedding(vocabulary_size + 1, embedding_size)
        self.name = name
        self.vocabulary_size = vocabulary_size

    def to_embeddings(self, idxs):
        if not torch.is_tensor(idxs):
            idxs = torch.tensor(idxs)
        with torch.no_grad():
            embeddings = self.lookup_table(idxs)
        return embeddings

class FeaturesEncoder:
    @staticmethod
    def get_instance(name: str, features_numeric, features_categoric = None):
        if features_categoric is None:
            filepath = os.path.join('.', 'lookup_tables', f'{name}-{"-".join(features_numeric)}.pickle')
        else:
            filepath = os.path.join('.', 'lookup_tables', f'{name}-{"-".join(features_numeric)}-{"-".join(features_categoric)}.pickle')
        if os.path.exists(filepath):
            with open(filepath, 'rb') as handle:
                encoder = pickle.load(handle)
            print(f'{name.capitalize()} loaded from {filepath}')
        else:
            encoder = FeaturesEncoder(features_numeric, features_categoric)
            with open(filepath, 'wb') as handle:
                pickle.dump(encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)
            print(f'{name.capitalize()} initialized and saved to {filepath}')
        return encoder
    
    def __init__(self, features_numeric, features_categoric ) -> None:
        self.features_numeric = features_numeric
        self.features_categoric = [] if features_categoric is None else list(features_categoric.keys())
        self.features_categoric_lookup_table = dict() if features_categoric is None else dict([
            (feature_name, CategoricalFeature(
                name=feature_name,
                vocabulary_size=vocabulary_size,
                embedding_size=embedding_size,
            ))
            for feature_name, (vocabulary_size, embedding_size) in features_categoric.items()
        ])

    def to_emebeddings(self, df: pd.DataFrame):
        numeric_features = df[self.features_numeric].to_numpy()
        df.drop(columns=self.features_numeric, inplace=True)
        numeric_features = torch.from_numpy(numeric_features).float()
        categoric_features = []
        for feature in self.features_categoric:
            encoder = self.features_categoric_lookup_table[feature]
            categoric_feature = df[feature].to_numpy()
            df.drop(columns=[feature], inplace=True)
            categoric_feature = torch.from_numpy(categoric_feature)
            feature_embedding = encoder.to_embeddings(categoric_feature)
            categoric_features.append(feature_embedding)
        # concat (numeric, ...categoric)
        return torch.cat((numeric_features, *categoric_features), 1)

In [4]:
class TrainTestMaker:

    def __init__(self) -> None:
        data = TrainTestMaker.read_all_data()
        self.data: pd.DataFrame = data
        user_features_numeric, item_features_numeric, item_features_categoric = TrainTestMaker.get_features_defs(data)
        self.user_encoder = FeaturesEncoder.get_instance('user', user_features_numeric)
        self.item_encoder = FeaturesEncoder.get_instance('item', item_features_numeric, item_features_categoric)

    @staticmethod
    def read_all_data() -> pd.DataFrame:
        paths = []
        dirs = os.listdir('normalized_train_data')
        for dir in dirs:
            filenames = os.listdir(f'normalized_train_data/{dir}')
            paths += [f'normalized_train_data/{dir}/{filename}' for filename in filenames]
        paths = sorted(paths)
        dfs = []
        for filepath in tqdm(paths, desc='Loading data'):
            df = pd.read_csv(filepath)
            df.drop(columns=['Unnamed: 0'], inplace=True)
            dfs.append(df)
        dfs = pd.concat(dfs)
        dfs.reset_index(drop=True, inplace=True)
        return dfs

    @staticmethod
    def load_encoders():
        features = [
            'user_id_hash',
            'target_id_hash',
            'syndicator_id_hash',
            'campaign_id_hash',
            'target_item_taxonomy',
            'placement_id_hash',
            'publisher_id_hash',
            'source_id_hash',
            'source_item_type',
            'browser_platform',
            'country_code',
            'region',
        ]
        column_encoders = dict()
        for feature in tqdm(features, desc='Loading encoders'):
            with open(f'./label_encoders/{feature}.pickle', 'rb') as handle:
                encoder = pickle.load(handle)
            column_encoders[feature] = encoder
        return column_encoders

    @staticmethod
    def get_features_defs(data):
        user_features_numeric = [
            'page_view_start_time',
            'user_recs',
            'user_clicks', 
            'user_target_recs',
        ]

        item_features_numeric = [
            'page_view_start_time',
            'empiric_calibrated_recs',
            'empiric_clicks',
        ]

        item_features_categoric = {
            # (vocabulary size, embedding size)
            'syndicator_id_hash': (data['syndicator_id_hash'].nunique(), 32),
            'campaign_id_hash': (data['campaign_id_hash'].nunique(), 32),
            'placement_id_hash': (data['placement_id_hash'].nunique(), 32),
            'target_item_taxonomy': (data['target_item_taxonomy'].nunique(), 8),
        }
        return user_features_numeric, item_features_numeric, item_features_categoric


    def _get_split(self):
        max_timestamp = self.data['page_view_start_time'].max()
        three_days_ago = (datetime.datetime.fromtimestamp(max_timestamp/1000) - datetime.timedelta(days=3)).replace(hour=0, minute=0, second=0, microsecond=0)
        three_days_ago = int(datetime.datetime.timestamp(three_days_ago) * 1000)
        return three_days_ago

    def get_trainset_df(self):
        threshold = self._get_split()
        train_set = self.data[self.data['page_view_start_time'] < threshold]
        train_set.reset_index(drop=True, inplace=True)
        return train_set

    def df_to_embeddings(self, df):
        batch_size = 1_000_000
        user_embeddings = []
        item_embeddings = []
        for i in range(0, len(df.index), batch_size):
            batch = df.iloc[i:i + batch_size]
            user_features_df = batch[self.user_encoder.features_numeric + self.user_encoder.features_categoric].copy(deep=True)
            del batch
            batch_user_embeddings = self.user_encoder.to_emebeddings(user_features_df)
            user_embeddings.append(batch_user_embeddings)
            del user_features_df
            batch = df.iloc[i:i + batch_size]
            item_features_df = batch[self.item_encoder.features_numeric + self.item_encoder.features_categoric].copy(deep=True)
            del batch
            batch_item_embeddings = self.item_encoder.to_emebeddings(item_features_df)
            item_embeddings.append(batch_item_embeddings)
            del item_features_df
        user_embeddings_concatenated = torch.cat(user_embeddings)
        del user_embeddings
        item_embeddings_concatenated = torch.cat(item_embeddings)
        del item_embeddings
        labels = df['is_click'].to_numpy().astype('int')
        labels_one_hot = np.zeros((labels.shape[0], 2))
        labels_one_hot[np.arange(labels.shape[0]), labels] = 1
        del labels
        return {
            'user': user_embeddings_concatenated,
            'item': item_embeddings_concatenated,
            'label': torch.from_numpy(labels_one_hot).float(),
        }
    
    def get_test_set_df(self):
        threshold = self._get_split()
        train_set = self.data[self.data['page_view_start_time'] < threshold]
        last_three_days = self.data[self.data['page_view_start_time'] >= threshold]
        not_cold_users_mask = last_three_days['user_id_hash'].isin(train_set['user_id_hash'])
        test_set_hot_users = last_three_days[not_cold_users_mask]
        test_set_cold_users = last_three_days[~not_cold_users_mask]
        test_set_hot_users.reset_index(drop=True, inplace=True)
        test_set_cold_users.reset_index(drop=True, inplace=True)
        return test_set_hot_users, test_set_cold_users

    def get_train_set(self, max_idx = None):
        print('Extracting Train set')
        train_set_df = self.get_trainset_df()
        print('Extraction completed')
        if max_idx is not None:
            train_set_df = train_set_df.iloc[: max_idx].copy()
        print('Converting to embeddings')
        train_set = self.df_to_embeddings(train_set_df)
        print('Convertion completed')
        return train_set
    
    def get_test_set(self):
        test_set_hot_users_df, test_set_cold_users_df = self.get_test_set_df()
        test_set_hot_users = self.df_to_embeddings(test_set_hot_users_df)
        del test_set_hot_users_df
        test_set_cold_users = self.df_to_embeddings(test_set_cold_users_df)
        del test_set_cold_users_df
        return test_set_hot_users, test_set_cold_users

In [5]:
data_maker = TrainTestMaker()
# train_set = data_maker.get_train_set(3_000_000)

Loading data:   0%|          | 0/69 [00:00<?, ?it/s]

User loaded from ./lookup_tables/user-page_view_start_time-user_recs-user_clicks-user_target_recs.pickle
Item loaded from ./lookup_tables/item-page_view_start_time-empiric_calibrated_recs-empiric_clicks-syndicator_id_hash-campaign_id_hash-placement_id_hash-target_item_taxonomy.pickle


In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.user_tower = nn.Sequential(
            nn.Linear(4, 4),
            nn.ReLU(),
            nn.Linear(4, 4),
            nn.ReLU(),
            nn.Linear(4, 4),
            nn.ReLU(),
        )

        self.item_tower = nn.Sequential(
            nn.Linear(107, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU()
        )
        self.classifier = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2),
            nn.ReLU()
        )
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        users, items = x
        users = self.user_tower(users)
        items = self.item_tower(items)
        users = nn.functional.pad(users, (0,60), value=1)
        aggregation = users * items
        return self.classifier(aggregation)


In [7]:
class TorchDataset(Dataset):
    def __init__(self, idxs, data):
        super().__init__()
        self.idxs = idxs
        self.data = data
        self.size = idxs.shape[0]

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        users_main_data = self.data['user']
        items_main_data = self.data['item']
        y_main_data = self.data['label']
        
        real_idxs = self.idxs[idx]
        
        users = users_main_data[real_idxs]
        items = items_main_data[real_idxs]
        y = y_main_data[real_idxs]
        
        return users, items, y

In [ ]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

class Training:

    def __init__(self, data_maker: TrainTestMaker, models_dir, data_size=None) -> None:
        self.data_maker = data_maker
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        # self.device = torch.device("cpu")
        data = data_maker.get_train_set(data_size)
        model = Model()
        model = model.to(self.device)
        # model = torch.compile(model)
        model.eval()
        self.model = model
        self.validation_percent = 0.05
        self.epochs = 20

        classes, class_occurances = data['label'].unique(return_counts=True)
        max_class_occurances = class_occurances.max()
        weights = (max_class_occurances / class_occurances).to(self.device)
        self._ce_weights = weights

        self.criterion = torch.nn.CrossEntropyLoss(weight=weights, label_smoothing=0.15)
        self.optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
        self._training_objects = dict()
        self.data = data
        self.models_dir = models_dir

    def _save(self, models_dir, epoch=None):
        if not os.path.isdir(models_dir):
            os.mkdir(models_dir)
        name = 'model-final.pt' if epoch is None else f'model-epoch-{epoch}.pt'
        torch.save(self.model, os.path.join(models_dir, name))


    def _build_loaders(self):
        users = self.data['user']

        permutation = np.random.permutation(len(users))
        validation_size = int(len(users) * self.validation_percent)
        validation_idxs = permutation[:validation_size]
        train_idxs = permutation[validation_size:]
        train_size = len(train_idxs)
        
        train_dataset = TorchDataset(train_idxs, self.data)
        validation_dataset = TorchDataset(validation_idxs, self.data)
        train_loader = DataLoader(train_dataset, batch_size=128, num_workers=1, shuffle=True)
        val_loader = DataLoader(validation_dataset, batch_size=128, num_workers=1, shuffle=False)
        return train_loader, train_size, val_loader, validation_size

    def _batch_train(self, data, stage):
        users, items, labels = data
        # mode to device/cuda
        users, items, labels = users.to(self.device), items.to(self.device), labels.to(self.device)
        # zero the parameter gradients
        self.optimizer.zero_grad()
        with torch.set_grad_enabled(stage == 'train'):
            # logits: The raw predictions from the last layer            
            logits = self.model((users, items))
            _, preds = torch.max(logits, 1)
            # return preds, labels
            # using CE criterion, thus input is the logits
            loss = self.criterion(logits, labels)
            if stage == 'train':
                loss.backward()
                self.optimizer.step()
        # statistics
        running_loss = loss.item() * users.size(0)
        running_corrects = torch.sum(preds == labels.argmax())
        return running_loss, running_corrects

    def _train(self, train_loader, train_size, val_loader, validation_size, models_dir):
        metrics = {
            'epochs': [],
            'loss': {
                'train': [],
                'validation': [],
            },
            'accuracy': {
                'train': [],
                'validation': [],
            }
        }
        print('Training the model')
        early_stopper = EarlyStopper(patience=3, min_delta=0.1)
        for epoch in tqdm(np.arange(self.epochs), desc=f'Epochs loop'):
            for stage in ['train', 'validation']:
                loss = self._epoch_train(stage, metrics, train_loader, train_size, val_loader, validation_size)
                if stage == 'validation':
                    validation_loss = loss
            self._save(models_dir, epoch)
            metrics['epochs'].append(epoch)
            if early_stopper.early_stop(validation_loss):
                print(f'Early stopping on epoch {epoch}')
                break
        print('Finished Training')
        self._save(models_dir)
        if not os.path.isdir(os.path.join('.', 'objects')):
            os.mkdir(os.path.join('.', 'objects'))
        with open(os.path.join(models_dir, 'training-metrics.pickle'), 'wb') as handle:
            pickle.dump(metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)
        return metrics


    def _epoch_train(self, stage, metrics, train_loader, train_size, val_loader, validation_size):
        epoch_loss = 0
        epoch_accuracy = 0
        if stage == 'train':
            self.model.train()
            loader = train_loader
            dataset_size = train_size
        else:
            self.model.eval()
            loader = val_loader
            dataset_size = validation_size
        running_loss = 0.0
        running_corrects = 0
        # for i, data in enumerate(loader, 0):
        for i, data in tqdm(enumerate(loader, 0), total=len(loader), desc=f'Stage {stage}: iterating over batches'):
            # return _batch_train(data, stage)
            current_loss, current_corrects = self._batch_train(data, stage)
            running_loss += current_loss
            running_corrects += current_corrects
        epoch_loss = running_loss / dataset_size
        epoch_accuracy = running_corrects.double() / dataset_size
        # accuracy is still tensor
        metrics['accuracy'][stage].append(epoch_accuracy.item())
        # loss in python native type number
        metrics['loss'][stage].append(epoch_loss)
        return epoch_loss


    def predict(self, dataset):
        test_dataset = TorchDataset(torch.arange(dataset['user'].size(0)), dataset)
        test_loader = DataLoader(test_dataset, batch_size=64, num_workers=1, shuffle=False)
        with torch.no_grad():
            self.model.eval()
            outputs = []
            probabilities_acc = []
            for users, items, labels in tqdm(test_loader, total=len(test_loader), desc=f'Prediction'):
                users, items = users.to(self.device), items.to(self.device)
                logits = self.model((users, items))
                logits = self.model.softmax(logits)
                probabilities = logits.cpu()
                predictions = logits.argmax(dim=1).cpu()
                outputs.append(predictions)
                probabilities_acc.append(probabilities)
            return torch.cat(outputs).numpy(), torch.cat(probabilities_acc).numpy()

    def show_metrics(self, metrics, dirpath):
        loss_df = pd.DataFrame(index=metrics['epochs'])
        loss_df['train'] = metrics['loss']['train']
        loss_df['validation'] = metrics['loss']['validation']
        accuracy_df = pd.DataFrame(index=metrics['epochs'])
        accuracy_df['train'] = metrics['accuracy']['train']
        accuracy_df['validation'] = metrics['accuracy']['validation']
        
        fig, axes = plt.subplots(2, 1, figsize=(10,10))
        axes = axes.flatten()
        
        sns.lineplot(data=loss_df, ax=axes[0])
        axes[0].grid()
        axes[0].set_ylabel('Loss')
        axes[0].set_xlabel('Epochs')

        sns.lineplot(data=accuracy_df, ax=axes[1])
        axes[1].grid()
        axes[1].set_ylabel('Accuracy')
        axes[1].set_xlabel('Epochs')
        fig.tight_layout()
        fig.savefig(os.path.join(dirpath, 'loss-accuracy-plots-during-training.png'))
        plt.close(fig)

    def run(self):
        train_loader, train_size, val_loader, validation_size = self._build_loaders()
        model = self.model.to(self.device)
        res = self._train(train_loader, train_size, val_loader, validation_size, self.models_dir)
        with open(os.path.join(self.models_dir, 'training-metrics.pickle'), 'rb') as handle:
            metrics = pickle.load(handle)
        self.show_metrics(metrics, self.models_dir)
        del self.data
        # collect garbage
        gc.collect()
        test_set_hot_users, test_set_cold_users = data_maker.get_test_set()
        predictions, probabilities = self.predict(test_set_hot_users)
        report = pd.DataFrame()
        
        report['ground-truth'] = test_set_hot_users['label'].numpy().astype('int').argmax(axis=1)
        report['predictions'] = predictions
        report['prediction-proba-0'] = probabilities[:, 0]
        report['prediction-proba-1'] = probabilities[:, 1]
        report.to_csv(os.path.join(self.models_dir, 'hot-users-predictions.csv'), index=False)

        del test_set_hot_users

        predictions, probabilities = self.predict(test_set_cold_users)
        report = pd.DataFrame()
        report['ground-truth'] = test_set_cold_users['label'].numpy().astype('int').argmax(axis=1)
        report['predictions'] = predictions
        report['prediction-proba-0'] = probabilities[:, 0]
        report['prediction-proba-1'] = probabilities[:, 1]
        report.to_csv(os.path.join(self.models_dir, 'cold-users-predictions.csv'), index=False)
        del test_set_cold_users


def run():
    current_datetime = datetime.datetime.now()
    date_str = current_datetime.strftime("%Y-%m-%d")
    time_str = current_datetime.strftime("%H-%M-%S")
    models_dir = os.path.join('.', 'models', f"torch-models-{date_str}T{time_str}")
    print(f'Directory {models_dir}')
    if not os.path.isdir(models_dir):
        os.mkdir(models_dir)
    train_manager = Training(
        data_maker,
        models_dir=models_dir,
        # data_size=3_000_000,
    )
    train_manager.run()
run()

Directory ./models/torch-models-2024-02-16T17-36-50
Extracting Train set
Extraction completed
Converting to embeddings
Convertion completed
Training the model


Epochs loop:   0%|          | 0/20 [00:00<?, ?it/s]

Stage train: iterating over batches:   0%|          | 0/180800 [00:00<?, ?it/s]

Stage validation: iterating over batches:   0%|          | 0/9516 [00:00<?, ?it/s]

Stage train: iterating over batches:   0%|          | 0/180800 [00:00<?, ?it/s]

...

Stage train: iterating over batches:   0%|          | 0/180800 [00:00<?, ?it/s]

Stage validation: iterating over batches:   0%|          | 0/9516 [00:00<?, ?it/s]

Stage train: iterating over batches:   0%|          | 0/180800 [00:00<?, ?it/s]

Stage validation: iterating over batches:   0%|          | 0/9516 [00:00<?, ?it/s]

Stage train: iterating over batches:   0%|          | 0/180800 [00:00<?, ?it/s]